In [1]:
import os
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import pandas as pd
import time
path = "alt.atheism/"

## Lab 2

Download and pre-process a large, publicly available text corpus. The preprocessing includes tokenization, normalization (e.g. convert to lowercase, remove non-alphanumeric chars, numbers, etc.). Output the preprocessing results as a text file, each line containing a single document (e.g. news article, sentence, yelp review, etc), with normalized tokens separated by a single white space. For example, the output file could look something like this:

_this is my first normalized sentence or document_

_this is my second sentence note that there are no numbers or punctuation_

_this is my third example note that all tokens are separated by a single white space_


In [2]:
text = []
for file in os.listdir(path):
    f = open(path + file, 'r', encoding = 'utf8', errors='ignore')
    text.append(f.read())
    f.close()

In [3]:
tokenized = []
for t in text:
    tokenized.append(nltk.word_tokenize(t))

In [4]:
normalized = []
for tokenized_text in tokenized:
    alphanum = [i.lower() for i in tokenized_text if i.isalpha()]
    normalized.append(alphanum)

In [5]:
with open('output.txt', 'w') as f:
    for doc in normalized:
        for token in doc:
            f.write(token+' ')
        f.write('\n')

Generate word2vec embeddings of your corpus using the original word2vec C implementation (https://code.google.com/archive/p/word2vec/ (Links to an external site.), https://github.com/tmikolov/word2vec (Links to an external site.)) or the python gensim implementation of the same algorithm (https://radimrehurek.com/gensim/models/word2vec.html (Links to an external site.)). Note that the gensim library can load/use word2vec embeddings generated with the original word2vec C implementation, which appears to be faster.

In [6]:
cbow_win5_emb100 = Word2Vec(normalized, size=100, window=5, min_count=3, sg=0, workers=4)

In [7]:
vector = cbow_win5_emb100.wv['computer']
vector

array([ 7.50116229e-01, -5.33902384e-02, -3.32647711e-01, -7.09465444e-01,
        3.30302417e-02,  9.18757319e-02, -1.21799827e-01, -2.23813251e-01,
       -1.32582396e-01,  8.81363273e-01,  3.31485689e-01,  1.53055191e-01,
       -5.61688364e-01, -4.53914642e-01,  2.02939585e-01, -1.16395831e+00,
        7.08374977e-01, -3.86702061e-01, -4.22304690e-01,  1.31783515e-01,
       -2.98752993e-01, -5.07760704e-01, -3.16906045e-03,  1.25914633e+00,
        6.23812806e-03,  2.30205521e-01,  1.80585086e-01,  4.97592449e-01,
       -2.18527988e-01, -4.60152090e-01,  8.76268864e-01,  4.30669822e-02,
        3.19028169e-01,  5.91184676e-01,  4.22868103e-01, -5.17717786e-02,
        5.96705496e-01, -5.77265799e-01,  1.35623083e-01,  5.53592503e-01,
        2.40148365e-01,  6.57165766e-01, -6.00041091e-01,  6.86857700e-02,
       -8.86012614e-01, -3.79629105e-01, -3.27166051e-01,  1.09146774e+00,
        9.13431227e-01, -1.16205618e-01,  3.45087983e-02,  2.12658361e-01,
        9.76695865e-02, -

## Homework 2
After completing the lab assignment, experiment with 2 or more sets of word2vec model parameters, for example, different embedding sizes, CBOW vs. skip-gram models, etc. Provide a paragraph or two of qualitative evaluation of your embeddings (no need to provide a quantitative evaluation). For example, you can evaluate the embeddings by hand-picking ~10 words and manually reviewing/evaluating their closest neighbors in terms of cosine similarity.

In [8]:
start = time.time()
cbow_win5_emb100 = Word2Vec(normalized, size=100, window=5, min_count=3, sg=0, workers=4)
print("word2vec using CBOW, window size 5, 100 embedding size takes %s"%(time.time()-start))

start = time.time()
cbow_win10_emb150 = Word2Vec(normalized, size=150, window=10, min_count=3, sg=0, workers=4)
print("word2vec using CBOW, window size 10, 150 embedding size takes %s"%(time.time()-start))

start = time.time()
skip_win5_emb100 = Word2Vec(normalized, size=100, window=5, min_count=3, sg=1, workers=4)
print("word2vec using skip-gram, window size 5, 100 embedding size takes %s"%(time.time()-start))

word2vec using CBOW, window size 5, 100 embedding size takes 1.9530611038208008
word2vec using CBOW, window size 10, 150 embedding size takes 1.988832950592041
word2vec using skip-gram, window size 5, 100 embedding size takes 2.966956853866577


In [9]:
def get_similar_most_words(model, target_words, num_of_words=5):
    res = {}
    for i in target_words:
        res[i] = model.wv.most_similar(i)[:num_of_words]
    return res

In [10]:
target_words = ['god', 'human', 'science', 'data', 'earth', 'play', 'physics', 'computer', 'study', 'myth']

In [11]:
pd.DataFrame(get_similar_most_words(cbow_win5_emb100, target_words))

god                               human  \
0       (fact, 0.9652177095413208)    (statements, 0.9898587465286255)   
1        (one, 0.9467852115631104)  (unreasonable, 0.9890013337135315)   
2   (question, 0.9437123537063599)       (without, 0.9864014387130737)   
3       (gods, 0.9388728141784668)          (life, 0.9859429001808167)   
4  (statement, 0.9352432489395142)       (contents, 0.983342170715332)   

                          science                               data  \
0     (basis, 0.9508533477783203)  (traditional, 0.9968911409378052)   
1     (steps, 0.9433700442314148)        (party, 0.9964227080345154)   
2  (alterius, 0.9377532005310059)          (per, 0.9959977865219116)   
3     (realm, 0.9364252090454102)  (invalidates, 0.9957599639892578)   
4   (atheism, 0.9351377487182617)     (goodness, 0.9956218600273132)   

                             earth                             play  \
0        (age, 0.9851472973823547)   (directly, 0.9976117610931396)   
1     (states, 0.9837836623191833)  (regarding, 0.9973920583724976)   
2    (planets, 0.9826770424842834)       (full, 0.9971146583557129)   
3  (condemned, 0.9823474287986755)         (sp, 0.9970144629478455)   
4  (disciples, 0.9811902046203613)     (unlike, 0.9966237545013428)   

                            physics                          computer  \
0  (supporting, 0.9892902374267578)        (dept, 0.9969635605812073)   
1        (rule, 0.9891491532325745)   (wisconsin, 0.9935089945793152)   
2        (imho, 0.9882904291152954)      (austin, 0.9915690422058105)   
3      (length, 0.9878376722335815)          (cs, 0.9915235638618469)   
4         (bug, 0.9868855476379395)  (technology, 0.9909390807151794)   

                            study                             myth  
0  (together, 0.9978960156440735)      (scale, 0.9945277571678162)  
1     (stand, 0.9971149563789368)      (seven, 0.9939987659454346)  
2    (unlike, 0.9964526295661926)  (reviewette, 0.993532657623291)  
3     (short, 0.9964014887809753)       (dark, 0.9933671355247498)  
4   (instead, 0.9963665008544922)     (couple, 0.9933009147644043)

In [12]:
pd.DataFrame(get_similar_most_words(cbow_win10_emb150, target_words))

god                         human  \
0      (existence, 0.9629144072532654)   (being, 0.9938219785690308)   
1           (fact, 0.9466567039489746)    (very, 0.9900756478309631)   
2         (belief, 0.9369016885757446)   (sides, 0.9892735481262207)   
3       (evidence, 0.9254992008209229)  (simply, 0.9888495802879333)   
4  (manifestation, 0.9245367050170898)    (work, 0.9876078367233276)   

                          science                                  data  \
0   (origins, 0.9624210000038147)       (violation, 0.9971508979797363)   
1     (basis, 0.9295337200164795)        (miracles, 0.9965916872024536)   
2       (hqd, 0.9100735187530518)             (nec, 0.9964849948883057)   
3  (waterloo, 0.9080314636230469)             (ban, 0.9957883358001709)   
4    (irving, 0.9015686511993408)  (incompleteness, 0.9957283735275269)   

                            earth                               play  \
0  (levelled, 0.9902923107147217)     (supports, 0.9984139800071716)   
1    (burden, 0.9890457391738892)  (translation, 0.9970253705978394)   
2    (states, 0.9890228509902954)   (assertions, 0.9968380928039551)   
3  (particle, 0.9885709285736084)     (theology, 0.9964954257011414)   
4       (esp, 0.9877588152885437)   (significant, 0.996310293674469)   

                           physics                          computer  \
0    (fiction, 0.9850502014160156)    (carnegie, 0.9936286211013794)   
1        (jkj, 0.9847123622894287)  (university, 0.9920341372489929)   
2          (k, 0.9827766418457031)         (mst, 0.9888726472854614)   
3     (length, 0.9818565845489502)    (keywords, 0.9870120882987976)   
4  (separates, 0.9814780950546265)   (computing, 0.9862393736839294)   

                             study                             myth  
0       (chip, 0.9975271224975586)      (eltnc, 0.9979461431503296)  
1  (recognize, 0.9971242547035217)       (mcvey, 0.997639000415802)  
2      (myths, 0.9961103200912476)        (eng, 0.9974782466888428)  
3  (pragmatic, 0.9953621625900269)  (leibowitz, 0.9972885847091675)  
4       (stop, 0.9951556921005249)    (whyalla, 0.9972806572914124)

In [13]:
pd.DataFrame(get_similar_most_words(skip_win5_emb100, target_words))

god                               human  \
0      (satan, 0.7217923998832703)  (intelligence, 0.9144315719604492)   
1      (exist, 0.6900628805160522)     (processes, 0.9011027812957764)   
2  (believing, 0.6897610425949097)   (individuals, 0.8980531096458435)   
3  (existence, 0.6873213052749634)         (result, 0.897000253200531)   
4     (exists, 0.6844085454940796)      (relative, 0.8929824233055115)   

                            science                                data  \
0     (percent, 0.6613577008247375)          (board, 0.948987603187561)   
1  (guaranteed, 0.6345124244689941)       (medical, 0.9306854605674744)   
2       (value, 0.6306748390197754)  (experimental, 0.9188069105148315)   
3    (computer, 0.6268389225006104)  (preservation, 0.9112571477890015)   
4     (mixture, 0.6199126839637756)      (northern, 0.9107797145843506)   

                           earth                             play  \
0    (orbit, 0.8593956232070923)       (pick, 0.9560073614120483)   
1   (starts, 0.8581326007843018)     (refute, 0.9553372859954834)   
2  (heavens, 0.8402745127677917)    (escapes, 0.9544968605041504)   
3    (night, 0.7679082751274109)  (literally, 0.9532182216644287)   
4     (luke, 0.7593796253204346)       (trap, 0.9523584842681885)   

                         physics                        computer  \
0  (pihatie, 0.7993094325065613)  (carnegie, 0.9562980532646179)   
1     (rule, 0.7873783707618713)      (dept, 0.9539111256599426)   
2   (boston, 0.7608155608177185)      (tech, 0.9470431208610535)   
3     (game, 0.7527147531509399)    (mellon, 0.9383187890052795)   
4    (ditto, 0.7299305200576782)   (virginia, 0.932227611541748)   

                               study                              myth  
0  (differences, 0.9717433452606201)      (illiad, 0.9435649514198303)  
1         (draw, 0.9691357016563416)   (relevance, 0.9393407106399536)  
2     (changing, 0.9681685566902161)       (jerry, 0.9353609085083008)  
3    (happiness, 0.9679674506187439)  (undisputed, 0.9350335001945496)  
4      (reflect, 0.9678720235824585)       (glory, 0.9347180128097534)

Read the required embeddings papers (word2vec, Bert, optionally Elmo) and compare the approaches in less than one page, preferably in a table format. You are free to decide what aspects to compare.  For example, you can include information such as: learning model details, summary of word context approaches, corpus size requirements, computational requirements, ease of installation/use of source code, date of publication and number of google scholar citations :), … etc.